# EDA 00 - Exploración de los datos del dataset principal

Nuestro dataset contiene diferentes ficheros con informaciones distintas. De esta forma, podremos crear diferentes sistemas de recomendación, de distinta complejidad y poder comparar los resultados de los mismos. De esta forma podremos aumentar nuestro entendimiento de los sistemas de recomendación y su funcionamiento. Encontramos los siguientes fichero en nuestro set de datos:

- **movies_metadata.csv** contiene los datos de las películas de manera general
- **credits.csv** contiene información relativa a los actores y director de las películas
- **keywords.csv** contiene información sobre palabras clave de las películas
- **ratings.csv** contiene datos relativos a diferentes usuarios y los rating que otorgan a las películas

Existen dos archivos más, que son para utilizarse con una porción del dataset más pequeña. En nuestro caso vamos a utilizar el dataset completo. En este _notebook_ trataremos únicamente con el fichero **movies_metadata.csv** y en diferentes _notebooks_ procesaremos los datos de los otros archivos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importación del _dataset_

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

DATA_PATH = os.getenv("FILES_PATH")

In [3]:
df = pd.read_csv(os.path.join(DATA_PATH, "CSV", "movies_metadata.csv"), low_memory=False)
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [6]:
df["genres"][:5]

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

Vemos que existen faltantes en varios campos, algunos de ellos no son de interés para el modelo (**homepage**, **belongs_to_collection**). También observamos que algunas de las entradas, como el género, están divididas en listas de diccionarios. Vamos a tener que formatear estas columnas para tener únicamente el género sin el id atribuido al mismo.

## Estudio de Características Individuales

De momento vamos a eliminar características que, a priori, no nos sean de utilidad. En la funcionalidad se ha propuesto tener el id de IMDB para realizar consultas a su API en las funcionalidades, pero vamos a eliminar esa característica en este _notebook_ porque en otros ficheros del set de datos figura la columna id. El csv resultante de este análisis será el que se utilice para la creación de los sistemas de recomendación.

In [8]:
df = df.drop(columns=["imdb_id"])

KeyError: "['imdb_id'] not found in axis"

Vemos que existen dos características relacionadas con el título: una de ellas es el título original de la película, en el idioma en el que se realizó. Como tenemos una característica de idioma original, hay redundancia, veamos aquellas entradas en las que el título original difiera del título otorgado a la película.

In [ ]:
df[df["original_title"] != df["title"]].loc[:,["original_title", "title"]].head()

Procedemos a eliminar la categoría de título original.

In [33]:
df = df.drop(columns=["original_title"])

Para realizar recomendaciones no vamos a utilizat el presupuesto o el beneficio. Son datos de interés para un análisis de las películas a nivel de gastos y beneficios. No obstante, las eliminamos porque carecen de información suficiente para darnos. Además, la mayoría de las películas carecen de estos datos en nuestro dataset.

In [34]:
df = df.drop(columns=["revenue", "budget"])

La columna de duración (_runtime_) es de tipo float, vamos a ver cuál es el rango de duración de las películas que tenemos:

In [35]:
df["runtime"].min(), df["runtime"].max()

(0.0, 1256.0)

In [36]:
df[df["runtime"] == 0].shape, df[df["runtime"].isnull()].shape

((1558, 20), (263, 20))

In [37]:
(len(df[df["runtime"] == 0]) + len(df[df["runtime"].isnull()])) / len(df)

0.040051906919456294

El 4% de las entradas carecen de tiempo de duración, por lo que eliminarlas no supondría un problema. De momento mantendremos la columna en su estado actual y más adelante vemos si rellenamos los faltantes con algún valor o eliminamos las entradas que carecen de duración. Vamos ahora con la característica de adulto:

In [38]:
df["adult"].value_counts()
df[df["adult"] == "False"]

,adult,belongs_to_collection,genres,homepage,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,[],NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


La mayor parte de las películas no son para adultos, por lo que tenemos 2 opciones: eliminar todas aquellas en las que adultos no sea False y eliminar la columna, o eliminar la columna directamente. En nuestro caso particular vamos a eliminar todas aquellas entradas en las que adulto es distinto de False:

In [39]:
df = df[df["adult"] == "False"].drop(columns=["adult"])

In [40]:
df.shape 

(45454, 19)

Ahora vamos a modificar la columna de fecha de lanzamiento (_release date_) para que sea de tipo datetime:

In [41]:
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

In [42]:
df["release_date"][:10]

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
5   1995-12-15
6   1995-12-15
7   1995-12-22
8   1995-12-22
9   1995-11-16
Name: release_date, dtype: datetime64[ns]

En la categoría de género vemos que las entradas figuran como listas de diccionarios. Vamos a utilizar una evaluación literal para cambiarlo a una lista de géneros:

In [43]:
from ast import literal_eval  # Utilizamos literal_eval para evaluar las cadenas de entradas del DataFrame como
                              # como listas de diccionarios para formatearlo mejor

eliminar_id_genre = lambda x: [genre["name"] for genre in literal_eval(x)] if isinstance(literal_eval(x), list) else []
df["genres"] = df["genres"].transform(func=eliminar_id_genre)

Vamos a comprobar cuántas entradas carecen de género:

In [44]:
df["genres"].value_counts()

genres
[Drama]                                                         5000
[Comedy]                                                        3620
[Documentary]                                                   2723
[]                                                              2442
[Drama, Romance]                                                1301
                                                                ... 
[Comedy, Crime, Mystery, Romance, Thriller]                        1
[Adventure, Animation, Action, Comedy, Family]                     1
[Mystery, Drama, Fantasy, Science Fiction, Thriller, Horror]       1
[Western, Music]                                                   1
[Family, Animation, Romance, Comedy]                               1
Name: count, Length: 4066, dtype: int64

Vemos que existen 2442 películas que carecen de géneros asociados. Esto no es un problema para un algortimo de _Collaborative Filtering_, pero lo tendremos en cuenta a futuro. Ahora vamos a analizar las rutas a los pósteres de las películas. Podría ser interesante que el usuario vea los pósteres en su front-end.

In [45]:
df[df["poster_path"].isnull()].shape

(386, 19)

In [46]:
df["poster_path"][:5]

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

Varias de las URL que aparecen llevan a enlaces que no tienen archivo, así que de momento, procedemos a eliminar la columna junto con la de homepage. Ésta última no nos proporciona información útil para la realización de nuestro recomendador.

In [47]:
df = df.drop(columns=["poster_path", "homepage"])

El estado de las películas no será relevante para su recomendación. No obstante, veamos cuántos valores tenemos en nuestro dataset:

In [48]:
df["status"].value_counts()

status
Released           45006
Rumored              230
Post Production       98
In Production         19
Planned               15
Canceled               2
Name: count, dtype: int64

Observamos que 2 películas fueron canceladas, entonces no se las recomendaríamos a ningún usuario. Para ello, las eliminaremos de nuestro set.

In [49]:
df = df[df["status"] != "Canceled"]
df.shape

(45452, 17)

La columna de tagline son comentarios breves sobre la película. Vemos que tenemos aproximadamente un 45% de faltantes, no obstante vamos a crear adelante una columna que contenga la sinopsis de la película junto con su tagline. Ahora, vamos a utilizar la misma estrategia con los idiomas hablados que la que utilizamos con el género:

In [50]:
df["spoken_languages"].head()

0             [{'iso_639_1': 'en', 'name': 'English'}]
1    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2             [{'iso_639_1': 'en', 'name': 'English'}]
3             [{'iso_639_1': 'en', 'name': 'English'}]
4             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, dtype: object

In [51]:
eliminar_id_idioma = lambda x: [lan["name"] for lan in literal_eval(x)] if isinstance(literal_eval(x), list) else []
df["spoken_languages"] = df["spoken_languages"].fillna("[]").transform(func=eliminar_id_idioma)

Utilizamos la misma estrategia de nuevo, pero para los valores de los países de producción:

In [52]:
eliminar_id_pais = lambda x: [country["name"] for country in literal_eval(x)] if isinstance(literal_eval(x), list) else []
df["production_countries"] = df["production_countries"].fillna("[]").transform(func=eliminar_id_pais)

Las compañías de producción son un detalle de interés para cinéfilos. No obstante, en nuestro sistema de recomendación no será un parámetro aunque pueda proporcionar similaridad entre películas de una misma compañía. Por ello, eliminamos la columna.

In [53]:
df = df.drop(columns=["production_companies"])

La columna de vídeo nos indica si existe algún vídeo presente sobre la película. Eliminamos esta característica por carecer de valor para nuestro sistema.

In [54]:
df = df.drop(columns=["video"])

Veamos cuántas columnas nos quedan en el DataFrame tras realizar un paso previo a un análisis más pormenorizado:

In [55]:
df.columns

Index(['belongs_to_collection', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_countries', 'release_date',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

## Limpieza Dataset

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45452 entries, 0 to 45465
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4490 non-null   object        
 1   genres                 45452 non-null  object        
 2   id                     45452 non-null  object        
 3   original_language      45441 non-null  object        
 4   overview               44498 non-null  object        
 5   popularity             45449 non-null  object        
 6   production_countries   45452 non-null  object        
 7   release_date           45367 non-null  datetime64[ns]
 8   runtime                45192 non-null  float64       
 9   spoken_languages       45452 non-null  object        
 10  status                 45368 non-null  object        
 11  tagline                20406 non-null  object        
 12  title                  45449 non-null  object        
 13  vote_a

La característica de la colección sería de interés en caso de que las películas tuvieran secuelas o precuelas. No obstante, si una película es secual de otra, la similaridad será alta, por lo que eliminamos esta característica.

El estado de las películas involucra todas menos las películas canceladas, por lo que ya no nos indica más información relevante para el modelo.

Los países de producción son de especial interés para conocer qué países producen más películas y realizar algunas gráficas para obtener información sobre el mundo del cine. En un sistema de recomendación puede sernos de utilidad para computar similaridad con otras películas, aunque en nuestro caso la vamos a eliminar.

Vamos a crear una columna que se llame _description_ que contenga la suma de la sinopsis junto con el tagline. Primero rellenamos los faltantes de tagline y creamos la nueva columna.

In [57]:
df["tagline"] = df["tagline"].fillna("")
df["description"] = df["overview"] + df["tagline"]
df = df.drop(columns=["tagline"])

In [58]:
df["description"].isnull().sum()

954

In [60]:
relevant_cols = ["genres", "id", "title", "overview", "description", "popularity", "vote_average", "vote_count"]

In [61]:
df = df[relevant_cols]

In [62]:
df.head()

,genres,id,title,overview,description,popularity,vote_average,vote_count
0,"[Animation, Comedy, Family]",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...",21.946943,7.7,5415.0
1,"[Adventure, Fantasy, Family]",8844,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...,17.015539,6.9,2413.0
2,"[Romance, Comedy]",15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...,11.7129,6.5,92.0
3,"[Comedy, Drama, Romance]",31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Cheated on, mistreated and stepped on, the wom...",3.859495,6.1,34.0
4,[Comedy],11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...,8.387519,5.7,173.0


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45452 entries, 0 to 45465
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45452 non-null  object 
 1   id            45452 non-null  object 
 2   title         45449 non-null  object 
 3   overview      44498 non-null  object 
 4   description   44498 non-null  object 
 5   popularity    45449 non-null  object 
 6   vote_average  45449 non-null  float64
 7   vote_count    45449 non-null  float64
dtypes: float64(2), object(6)
memory usage: 3.1+ MB


En el caso de overview vemos que tenemos unas 1000 películas que carecen de reseña. Dado que no podríamos rellenarlas de una manera completa, vamos a eliminar del set todas las películas que carecen de reseña.

In [64]:
df = df[df["overview"].notnull()]

In [65]:
df.isnull().sum()

genres          0
id              0
title           3
overview        0
description     0
popularity      3
vote_average    3
vote_count      3
dtype: int64

Eliminando las películas sin reseña obtenemos un dataset que apenas tiene faltantes. Vamos a rellenar esos datos numéricos faltantes con la mediana y exportaremos este dataset limpio para realizar nuestro primer recomendador en otro _Notebook_.

In [66]:
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")

In [67]:
df.isnull().sum()

genres          0
id              0
title           3
overview        0
description     0
popularity      3
vote_average    3
vote_count      3
dtype: int64

In [68]:
df.fillna({
    "popularity": df["popularity"].median(),
    "vote_average": df["vote_average"].median(),
    "vote_count": df["vote_count"].median()
}, inplace=True)

In [69]:
len(df), df.columns

(44498,
 Index(['genres', 'id', 'title', 'overview', 'description', 'popularity',
        'vote_average', 'vote_count'],
       dtype='object'))

In [72]:
df = df.drop_duplicates("id")
df = df.drop_duplicates("title")
df.shape

(41362, 8)

In [73]:
df.to_csv(os.path.join(DATA_PATH, "CSV", "cleaned_movies.csv"), index=False)